## 영화 추천 시스템(tfidf, cos sim) 
 - 영화제목을 입력하면 줄거리가 가장 비슷한 영화

In [38]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv("movies_metadata.csv",low_memory=False) # 대용량 데이터를 사용할땐 False로 놓아라.
# 데이터 읽을 때 데이터 컬럼의 타입을 예측하도록 되어있어서 시간이 조금 걸린다
# 그걸 하지 않도록 하는게 low_memory = False

In [6]:
data.shape

(45466, 24)

In [11]:
data = data.head(20000)
data.shape

(20000, 24)

In [14]:
data['overview']

0        Led by Woody, Andy's toys live happily in his ...
1        When siblings Judy and Peter discover an encha...
2        A family wedding reignites the ancient feud be...
3        Cheated on, mistreated and stepped on, the wom...
4        Just when George Banks has recovered from his ...
                               ...                        
19995    Dissidents in a French colony attack a police ...
19996    A young mother Nina and her son Enzo find them...
19997    An in-depth analysis of the relationship betwe...
19998    Follows the life and work of animator Lotte Re...
19999    An in-depth look at the genesis, production, a...
Name: overview, Length: 20000, dtype: object

In [16]:
# 전처리
data['overview'].isnull().sum()

135

In [20]:
# na → ''로 치환
data['overview'] = data['overview'].fillna('')

In [21]:
data['overview'].isnull().sum()

0

In [22]:
# tfidf
from sklearn.feature_extraction.text import TfidfVectorizer

In [31]:
tfidfMat = TfidfVectorizer(stop_words='english')
tfidfMat = tfidfMat.fit_transform(data['overview'])

In [33]:
tfidfMat.shape

(20000, 47487)

In [39]:
# 첫번째 영화의 tfidf
print(tfidfMat[0].toarray())
print(np.max(tfidfMat[0].toarray())) # 0.536643 → 굉장히 중요한 단어야.
tfidfMat[0].toarray().shape

[[0. 0. 0. ... 0. 0. 0.]]
0.5036643709879097


(1, 47487)

In [42]:
# 0.536643에 해당되는 단어? index번호?
idx=tfidfMat[0].toarray().argmax() # 가장 큰 값에 해당하는 argument index 출력
idx

6290

In [43]:
vocab = TfidfVectorizer(stop_words='english').fit(data['overview']).vocabulary_
vocab

{'led': 24361,
 'woody': 46617,
 'andy': 2051,
 'toys': 43131,
 'live': 24957,
 'happily': 18727,
 'room': 36185,
 'birthday': 4736,
 'brings': 5775,
 'buzz': 6290,
 'lightyear': 24759,
 'scene': 37166,
 'afraid': 1254,
 'losing': 25229,
 'place': 32291,
 'heart': 19047,
 'plots': 32424,
 'circumstances': 7974,
 'separate': 37778,
 'owner': 30780,
 'duo': 12920,
 'eventually': 14393,
 'learns': 24325,
 'aside': 2811,
 'differences': 11645,
 'siblings': 38509,
 'judy': 22536,
 'peter': 31851,
 'discover': 11888,
 'enchanted': 13728,
 'board': 5054,
 'game': 16798,
 'opens': 30240,
 'door': 12442,
 'magical': 25725,
 'world': 46664,
 'unwittingly': 44575,
 'invite': 21663,
 'alan': 1495,
 'adult': 1135,
 'trapped': 43304,
 'inside': 21286,
 '26': 430,
 'years': 46965,
 'living': 24973,
 'hope': 19854,
 'freedom': 16355,
 'finish': 15562,
 'proves': 33496,
 'risky': 35885,
 'running': 36455,
 'giant': 17265,
 'rhinoceroses': 35691,
 'evil': 14422,
 'monkeys': 28035,
 'terrifying': 42245,


In [46]:
[key for key, value in vocab.items() if value == idx] # 최빈 단어

['buzz']

In [47]:
tfidfMat # tfidf 행렬

# 코사인 유사도 구해야해.

<20000x47487 sparse matrix of type '<class 'numpy.float64'>'
	with 535909 stored elements in Compressed Sparse Row format>

### 코사인 유사도

In [48]:
# 코사인 유사도 
from sklearn.metrics.pairwise import linear_kernel

In [50]:
cos_sim = linear_kernel(tfidfMat, tfidfMat)
cos_sim.shape

(20000, 20000)

In [54]:
# 모든 영화와의 코사인 유사도
cos_sim[0].shape

(20000,)

In [57]:
print(data.index)
data['title']

RangeIndex(start=0, stop=20000, step=1)


0                                                Toy Story
1                                                  Jumanji
2                                         Grumpier Old Men
3                                        Waiting to Exhale
4                              Father of the Bride Part II
                               ...                        
19995                                            Rebellion
19996                                           Versailles
19997                                      Two in the Wave
19998    Lotte Reiniger: Homage to the Inventor of the ...
19999    RKO Production 601: The Making of 'Kong, the E...
Name: title, Length: 20000, dtype: object

In [61]:
indices = pd.Series(data.index,index=data['title']).drop_duplicates() # 중복제거!!!

In [62]:
indices['Jumanji']

1

In [ ]:
# Jumanji와 코사인유사도가 가장 높은 20편의 영화제목 출력(내림차순)

In [63]:
from sklearn.metrics.pairwise import linear_kernel

cos_sim=linear_kernel(tfidfMat, tfidfMat)
indices=pd.Series(data.index, index=data['title']).drop_duplicates()

# Jumanji영화 index
movie='Jumanji'
movie_idx=indices[movie]

# Jumanji영화와 코사인 유사도가 가장 높은 20편 영화 찾기
cos_sim_movies=list(enumerate(cos_sim[movie_idx]))
cos_sim_movies=sorted(cos_sim_movies, key=lambda x: x[1],reverse=True)[1:21]

# 영화 제목 찾기
cos_sim_movies_titles=[indices.index[x[0]] for x in cos_sim_movies]

# 영화 제목 20개 출력
print(f'{movie} 의 줄거리와 가장 코사인 유사도가 높은 영화 20편 : ')
print(cos_sim_movies_titles)

Jumanji 의 줄거리와 가장 코사인 유사도가 높은 영화 20편 : 
['Brainscan', 'Quintet', 'The Dark Angel', 'Word Wars', 'The Mindscape of Alan Moore', 'DeVour', 'Masques', 'Poolhall Junkies', 'Wreck-It Ralph', 'eXistenZ', 'Stay Alive', 'Rhinoceros', 'The Innocent Sleep', 'Nirvana', 'The Wave', "Grandma's Boy", 'The Last of Sheila', 'Gamer', 'Battlefield Baseball', 'Le Pont du Nord']
